<a href="https://colab.research.google.com/github/ravitata/tensorflow2/blob/master/tf-data-learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print (tf.__version__)

2.3.0


In [2]:
#output is scalar
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5]) # 1D
print(dataset)

<TensorSliceDataset shapes: (), types: tf.int32>


In [3]:
#output is 1D tensor
dataset = tf.data.Dataset.from_tensor_slices([[1,2],[3,4],[5, 6]]) # 2D 3*2
print(dataset)
print("")
print(list(dataset.as_numpy_iterator()))
print("")
for ele in dataset:
  print(ele)
  print(ele.numpy())

<TensorSliceDataset shapes: (2,), types: tf.int32>

[array([1, 2], dtype=int32), array([3, 4], dtype=int32), array([5, 6], dtype=int32)]

tf.Tensor([1 2], shape=(2,), dtype=int32)
[1 2]
tf.Tensor([3 4], shape=(2,), dtype=int32)
[3 4]
tf.Tensor([5 6], shape=(2,), dtype=int32)
[5 6]


In [4]:
dataset = tf.data.Dataset.from_tensor_slices(tf.random.normal([3, 2])) #1D tensor
print(dataset)
print("")
print(dataset.element_spec)
print("")
print(dataset.cardinality()) #Return number of elements

<TensorSliceDataset shapes: (2,), types: tf.float32>

TensorSpec(shape=(2,), dtype=tf.float32, name=None)

tf.Tensor(3, shape=(), dtype=int64)


In [10]:
dataset = tf.data.Dataset.from_tensor_slices(
    (
        [[1,2], [3,4], [5,6]],
        ['A', 'B','C']
    )
)
print(dataset.element_spec)
print("")

for elem in dataset:
  print(elem)
print("")

for elem in dataset.take(2):
  print(elem)

(TensorSpec(shape=(2,), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>, <tf.Tensor: shape=(), dtype=string, numpy=b'A'>)
(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4], dtype=int32)>, <tf.Tensor: shape=(), dtype=string, numpy=b'B'>)
(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([5, 6], dtype=int32)>, <tf.Tensor: shape=(), dtype=string, numpy=b'C'>)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>, <tf.Tensor: shape=(), dtype=string, numpy=b'A'>)
(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4], dtype=int32)>, <tf.Tensor: shape=(), dtype=string, numpy=b'B'>)


In [40]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
print(dataset.element_spec)
print('')
print(dataset.cardinality())


(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))

tf.Tensor(50000, shape=(), dtype=int64)


In [59]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [49]:
inputs = Input((32,32,3))
h = Conv2D(8, kernel_size=3,activation='relu')(inputs)
h = Conv2D(18, kernel_size=3, activation='relu')(h)
h = Flatten()(h)
outputs = Dense(1, activation='sigmoid')(h)
model = Model(inputs=inputs, outputs=outputs)

rmsprop = RMSprop(learning_rate=1e-3)
model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
model.summary()

Model: "functional_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 30, 30, 8)         224       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 18)        1314      
_________________________________________________________________
flatten_3 (Flatten)          (None, 14112)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 14113     
Total params: 15,651
Trainable params: 15,651
Non-trainable params: 0
_________________________________________________________________


In [51]:
'''
img_datagen = ImageDataGenerator()
dataset = tf.data.Dataset.from_generator(img_datagen.flow,
                                         args = [x_train, y_train],
                                         output_types = (tf.float32, tf.int32),
                                         output_shapes = ([32,32,32,3], [32,10])
                                         ) 
'''                                    

'\nimg_datagen = ImageDataGenerator()\ndataset = tf.data.Dataset.from_generator(img_datagen.flow,\n                                         args = [x_train, y_train],\n                                         output_types = (tf.float32, tf.int32),\n                                         output_shapes = ([32,32,32,3], [32,10])\n                                         ) \n'

In [52]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [53]:
# Calculate the training generator and test generator steps per epoch
batch_size=32
train_steps_per_epoch = x_train.shape[0] // batch_size
#val_steps = val_generator.n // val_generator.batch_size
#print(train_steps_per_epoch, val_steps)
print(train_steps_per_epoch)

1562


In [55]:
print(dataset.element_spec)

(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))


In [64]:
dataset = dataset.batch(64)

In [65]:

model.fit(dataset, epochs=1, steps_per_epoch = train_steps_per_epoch)

1562/1562 [==============================] - 6s 4ms/step - loss: 11.3882 - accuracy: 0.1161
